In [6]:
# Challenge 1: Supervised vs Unsupervised Learning
# Goal: Compare Random Forest and K-Means with same general settings to exceed baseline results

# Import necessary packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import kagglehub

# General setting - do not change TEST_SIZE
RANDOM_SEED = 42
TEST_SIZE = 0.3

# Load dataset & prepare data
# Load dataset (from kagglehub)
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data['Class'] = data['Class'].astype(int)

# Prepare data
data = data.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

# Fraud/Non-Fraud Transactions Analysis
fraud = data[data['Class'] == 1]
nonfraud = data[data['Class'] == 0]
print(f'Fraudulent:{len(fraud)}, non-fraudulent:{len(nonfraud)}')
print(f'the positive class (frauds) percentage: {len(fraud)/(len(fraud) + len(nonfraud))}')
print(f'(len(fraud)/(len(fraud) + len(nonfraud)))*100:.3f%')

# Supervised Learning (Random Forest)
X = np.asarray(data.iloc[:, ~data.columns.isin(['Class'])])
Y = np.asarray(data.iloc[:, data.columns == 'Class'])

# Split training set and data set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

# Build Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED)
rf_model.fit(X_train, y_train)

# Define evaluation function
def evaluation(y_true, y_pred, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f'\n{model_name} Evaluation:')
    print('===' * 15)
    print(f'         Accuracy:', accuracy)
    print(f' Precision Score:', precision)
    print(f'    Recall Score:', recall)
    print(f'       F1 Score:', f1)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

# Predict and print result
y_pred = rf_model.predict(X_test)
evaluation(y_test, y_pred, model_name="Random Forest")

# Unsupervised Learning (KMeans)
# Extract features and labels
X = np.asarray(data.drop(columns=['Class']))
y = np.asarray(data['Class'])

# Split the dataset into training and testing sets (with stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Select a small sample of normal (non-fraud) data for unsupervised training
n_x_train = X_train[y_train == 0]
n_x_train = n_x_train[:1000]

# Find optimal number of clusters using silhouette score
scores = []
for k in range(2, 5):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=RANDOM_SEED)
    kmeans.fit(n_x_train)
    score = silhouette_score(n_x_train, kmeans.labels_)
    scores.append(score)

optimal_k = np.argmax(scores) + 2
kmeans = KMeans(n_clusters=optimal_k, init='k-means++', random_state=RANDOM_SEED)
kmeans.fit(n_x_train)

# Predict on test set
y_pred_test = kmeans.predict(X_test)

# Align labels with ground truth
def align_labels(y_true, y_pred, n_clusters):
    labels = np.zeros_like(y_pred)
    for i in range(n_clusters):
        mask = (y_pred == i)
        if np.sum(mask) > 0:
            labels[mask] = np.bincount(y_true[mask]).argmax()
        else:
            labels[mask] = 0  # Default to normal class
    return labels

y_pred_aligned = align_labels(y_test, y_pred_test, optimal_k)

# Evaluate K-Means model
evaluation(y_test, y_pred_aligned, model_name="KMeans (Unsupervised)")

print("\n" + "="*50)
print("CHALLENGE 1 COMPLETED")
print("Comparison of Random Forest (Supervised) vs K-Means (Unsupervised)")
print("="*50)

c:\Users\User\NTCU-Machine-Learning\ACS111703 ex1\fraud_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fraudulent:492, non-fraudulent:284315
the positive class (frauds) percentage: 0.001727485630620034
(len(fraud)/(len(fraud) + len(nonfraud)))*100:.3f%


c:\Users\User\NTCU-Machine-Learning\ACS111703 ex1\fraud_env\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Random Forest Evaluation:
         Accuracy: 0.9996371850239341
 Precision Score: 0.9411764705882353
    Recall Score: 0.8235294117647058
       F1 Score: 0.8784313725490196

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.94      0.82      0.88       136

    accuracy                           1.00     85443
   macro avg       0.97      0.91      0.94     85443
weighted avg       1.00      1.00      1.00     85443


KMeans (Unsupervised) Evaluation:
         Accuracy: 0.9987242957293166
 Precision Score: 0.782608695652174
    Recall Score: 0.36486486486486486
       F1 Score: 0.4976958525345622

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.78      0.36      0.50       148

    accuracy                           1.00     85443
   macro avg       0.89      0.68      0.75     8